In [26]:
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve, classification_report

## Defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'data-445-bucket-callaghan'
bucket = s3.Bucket(bucket_name)

## Defining the csv file
file_key = 'telecom_churn.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## 1. Using the pandas library to read the csv data file and create a data-frame called churn_data

churn_data = pd.read_csv(file_content_stream)

churn_data.head()

,Churn,AccountWeeks,ContractRenewal,DataPlan,DataUsage,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
0,0,128,1,1,2.7,1,265.1,110,89.0,9.87,10.0
1,0,107,1,1,3.7,1,161.6,123,82.0,9.78,13.7
2,0,137,1,0,0.0,0,243.4,114,52.0,6.06,12.2
3,0,84,0,0,0.0,2,299.4,71,57.0,3.10,6.6
4,0,75,0,0,0.0,3,166.7,113,41.0,7.42,10.1


In [27]:
## 2. Reporting the relative frequency table of the Churn variable

churn_data['Churn'].value_counts() / churn_data.shape[0]

0    0.855086
1    0.144914
Name: Churn, dtype: float64

In [28]:
## 3. Using AccountWeeks, ContractRenewal, CustServCalls, MonthlyCharge, and DayMins as the predictor variables, 
## and Churn as the target variable, splitting the data into two data-frames (taking into account the proportion 
## of 0s and 1s): train (80%) and test (20%).


## Defining the input and target variables
X = churn_data[['AccountWeeks', 'ContractRenewal', 'CustServCalls', 'MonthlyCharge', 'DayMins']]
Y = churn_data['Churn']

## Splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)


## Using tree based methods, therefore we do not need to standardize the input data

In [29]:
## 4. Using the train data-frame to build a random forest classification model with 500 trees and
## the maximum depth of each tree equal to 3

## Building the model
rf_md = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_train, Y_train)

## Predicting on the test set
rf_preds = rf_md.predict_proba(X_test)[:, 1]

## Computing the ROC-Curve
fpr, tpr, threshold = roc_curve(Y_test, rf_preds)

## Creating a data frame to store ROC-Curve results
cutoffs = pd.DataFrame({'False_Positive': fpr, 'True_Positive': tpr, 'Cutoff': threshold})

## Calculating the Euclidean distance between each point and our optimal model (0,1)
cutoffs['True_Positive_Minus_1'] = cutoffs['True_Positive'] - 1
cutoffs['Distance'] = np.sqrt(cutoffs['False_Positive']**2 + cutoffs['True_Positive_Minus_1']**2)  ## Euclidean distance formula

## Sorting the data frame based on Euclidean distance
cutoffs = cutoffs.sort_values('Distance', ascending = True).reset_index(drop = True)

## Changing likelihoods to labels
rf_preds = np.where(rf_preds < cutoffs['Cutoff'][0], 0, 1)

## Using the optimal cutoff value to create the classification report
print(classification_report(Y_test, rf_preds))

              precision    recall  f1-score   support

           0       0.97      0.83      0.89       570
           1       0.46      0.86      0.60        97

    accuracy                           0.83       667
   macro avg       0.72      0.84      0.75       667
weighted avg       0.90      0.83      0.85       667

